In [10]:
import pandas as pd
import os
import numpy as np
from math import radians, cos, sin, asin, sqrt
import datetime
# import geopy
# import geopy.distance

In [2]:
ytd_data_path = os.path.join('..', 'data','NYPD_Arrest_Data__Year_to_Date_.csv')
crime_df = pd.read_csv(ytd_data_path, nrows=50)
print(crime_df.shape)
crime_df.head()

(50, 19)


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,222243929,12/25/2020,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,B,40,0,25-44,M,BLACK,1007453,233952,40.808798,-73.916184,POINT (-73.91618413199996 40.80879780500004)
1,222115778,12/22/2020,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1601001,F,M,33,0,18-24,M,BLACK HISPANIC,1001456,247485,40.845956,-73.937813,POINT (-73.93781267199995 40.84595574000008)
2,222090516,12/21/2020,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,Q,106,0,25-44,M,BLACK,1028605,187930,40.682398,-73.840079,POINT (-73.84007936899997 40.68239828900005)
3,221653577,12/10/2020,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,Q,103,0,18-24,M,BLACK,1039602,190480,40.689336,-73.800409,POINT (-73.80040882999998 40.68933550400004)
4,221599602,12/09/2020,729,"FORGERY,ETC.,UNCLASSIFIED-FELO",113,FORGERY,PL 1702500,F,S,120,0,18-24,M,BLACK,949767,170539,40.634708,-74.124241,POINT (-74.12424111699994 40.634707811000055)


In [3]:
full_df_path: str = os.path.join('..', 'data', 'NYPD_Arrests_Data__Historic_.csv')
sample_df = pd.read_csv(full_df_path, parse_dates=[1], infer_datetime_format=True, nrows=5)
chunksize: int = 10 ** 5
column_names= list(sample_df)

    
def parse_crime_df(file_path: str, year: int, chunksize: int) -> pd.DataFrame:
    """Reads the csv in `chunksize` pieces.  Filters csv to only keep crimes committed in `year`.
       Returns a pandas dataframe with all crimes from the selected year.
    """
    base_df = pd.DataFrame(columns=column_names)
    with pd.read_csv(full_df_path, parse_dates=[1], infer_datetime_format=True, chunksize=chunksize) as reader:
        for chunk in reader:
            filtered_df = chunk[chunk.ARREST_DATE.dt.year == year].copy()
            base_df = base_df.append(filtered_df)
    return base_df

In [4]:
print(f'Start Time: {datetime.datetime.now()}\n')

filtered_2018_df = parse_crime_df(file_path=full_df_path, year=2018, chunksize=chunksize)

print(f'Finish Time: {datetime.datetime.now()}\n')

print(f'DF shape: {filtered_2018_df.shape}')

Start Time: 2021-05-06 14:03:57.420990

Finish Time: 2021-05-06 14:04:23.200164

DF shape: (246773, 19)


In [5]:
print(f'Start Time: {datetime.datetime.now()}\n')

filtered_2019_df = parse_crime_df(file_path=full_df_path, year=2019, chunksize=chunksize)

print(f'Finish Time: {datetime.datetime.now()}\n')

print(f'DF shape: {filtered_2019_df.shape}')

Start Time: 2021-05-06 14:04:23.206939

Finish Time: 2021-05-06 14:04:48.364606

DF shape: (214617, 19)


In [6]:
print(f'Start Time: {datetime.datetime.now()}\n')

filtered_2017_df = parse_crime_df(file_path=full_df_path, year=2017, chunksize=chunksize)

print(f'Finish Time: {datetime.datetime.now()}\n')

print(f'DF shape: {filtered_2017_df.shape}')

Start Time: 2021-05-06 14:04:48.371113

Finish Time: 2021-05-06 14:05:14.431347

DF shape: (286225, 19)


In [15]:
filtered_2017_df.reset_index(inplace=True, drop=True)

filtered_2018_df.reset_index(inplace=True, drop=True)

filtered_2019_df.reset_index(inplace=True, drop=True)

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,192799737,2019-01-26,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
1,193260691,2019-02-06,NaN,NaN,NaN,NaN,PL 2203400,F,M,14,0,25-44,M,UNKNOWN,986685.0,215375.0,40.757839,-73.991212,POINT (-73.99121211099998 40.75783900300007)
2,196324211,2019-04-23,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,K,77,0,45-64,M,BLACK HISPANIC,1003606.0,185050.0,40.674583,-73.930222,POINT (-73.93022154099998 40.67458330800008)
3,196785901,2019-05-04,175.0,"SEXUAL ABUSE 3,2",233.0,SEX CRIMES,PL 13052A1,M,B,50,0,25-44,M,BLACK,1011257.0,261130.0,40.883383,-73.902333,POINT (-73.90233330899997 40.88338257900006)
4,197554056,2019-05-23,175.0,"SEXUAL ABUSE 3,2",233.0,SEX CRIMES,PL 13052A1,M,M,26,0,45-64,F,BLACK,996241.0,236149.0,40.814850,-73.956682,POINT (-73.95668184799996 40.81485028900005)


In [16]:
ny_addresses = pd.read_csv(os.path.join('..', 'data', 'city_of_new_york.csv'))
ny_addresses.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,-73.979408,40.756086,560,5 AVE,NaN,NaN,NaN,NaN,10036.0,1030290,3f160cf4e81705e8
1,-73.994266,40.701937,25,COLUMBIA HTS,NaN,NaN,NaN,NaN,11201.0,3002257,5d837357404d96f1
2,-73.918882,40.712780,1903,FLUSHING AVE,NaN,NaN,NaN,NaN,11385.0,89223,6a1943b8f15785d1
3,-73.986376,40.778571,205,W END AVE,NaN,NaN,NaN,NaN,10023.0,1027498,183423d02688b1b4
4,-73.912696,40.884183,3205,ARLINGTON AVE,NaN,NaN,NaN,NaN,10463.0,5178668,d0f8835b1818545c


In [ ]:
# Try 1: too slow

# def get_zipcode(df, geolocator, lat_field, lon_field):
#     location = geolocator.reverse((df[lat_field], df[lon_field]))
#     return location.raw['address']['postcode']

# geolocator = geopy.Nominatim(user_agent='umads_591')

# print(f'Time: {datetime.datetime.now()}\n')
# zipcodes = crime_df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')
# print(f'Time: {datetime.datetime.now()}\n')

In [ ]:
# Try 2: still too slow
# brute force
# 
# def get_crime_zip_codes(crime_df: pd.DataFrame, address_lookup_df: pd.DataFrame, min_distance_threshold: float = 1.0, 
#                         print_rate:int = 1, verbose: bool = False, save_checkpoints:bool = False):
#     """"""
#     for crime_idx, crime in crime_df.iterrows():
#         if crime_idx % print_rate == 0:
#             print(f'Starting on crime index {crime_idx}')
#             print(f'Time: {datetime.datetime.now()}\n')
#         if crime_idx % 10000 == 0 and save_checkpoints:
#             print(f'Checkpointing on crime index {crime_idx}')
#             crime_df.to_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'), index=False)
#         crime_lat = crime.Latitude
#         crime_long = crime.Longitude
#         for address_idx, address in address_lookup_df.iterrows():
#             address_lat = address.LAT
#             address_long = address.LON
#             distance = geopy.distance.distance((crime.Latitude, crime.Longitude), (address.LAT, address.LON)).miles
#             if verbose:
#                 print(f'Address: {address_idx} Distance: {distance}')
#             if distance < min_distance_threshold:
#                 crime_df.loc[crime_idx, 'crime_zip_code'] = address.POSTCODE
#                 break
#     return crime_df

# get_crime_zip_codes(crime_df=filtered_2018_df, address_lookup_df=ny_addresses, min_distance_threshold=1.0, print_rate=1, verbose=False)

In [ ]:
# SAVE BUT DOES NOT WORK
# Fast but no correct

# import numpy as np
# crime_coords = np.array(filtered_2018_df[['Latitude', 'Longitude']])
# address_coords = np.array(ny_addresses[['LAT', 'LON']])

# # crime_df_shape = crime_coords.shape[0]

# max_cos_idx = []

# for crime in crime_coords[:5]:
        
#     cosine_similarity = np.dot(crime,address_coords.T)/linalg.norm(crime)/linalg.norm(address_coords)
#     max_cos_idx.append(cosine_similarity.argmax())

In [ ]:
# filtered_2018_df_sample = filtered_2018_df.loc[:10,:].copy()
# get_crime_zip_codes(crime_df=filtered_2018_df_sample, address_lookup_df=ny_addresses, min_distance_threshold=1.0, print_rate=1, verbose=False)

In [ ]:
# updated_2018_df = get_crime_zip_codes(crime_df=filtered_2018_df, address_lookup_df=ny_addresses, min_distance_threshold=1.0, 
#                                       print_rate=1000, verbose=False, save_checkpoints=True)

# updated_2018_df.to_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'), index=False)

In [ ]:
# updated_2018_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'))
# updated_2018_df[updated_2018_df.crime_zip_code.isna()].head()

In [17]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # radius of earth in miles
    dist = 3956 * c
    return dist

In [18]:
# testing, put into function later

# %%time

# crime_coords = np.array(filtered_2018_df[['Latitude', 'Longitude']])
# address_coords = np.array(ny_addresses[['LAT', 'LON']])

# max_cos_idx = []
# zip_code_list = []

# print(f'Start Time: {datetime.datetime.now()}\n')

# for crime in crime_coords:
#     for idx, address in enumerate(address_coords):
#         h_dist = haversine(crime[0], crime[1], address[0], address[1])
#         if h_dist < 0.5:
#             max_cos_idx.append(idx)
#             zip_code_list.append(int(ny_addresses.loc[idx, 'POSTCODE']))
#             break
            
# print(f'Finish Time: {datetime.datetime.now()}\n')

In [32]:
def get_crime_zips(crime_coords, address_coords, ny_addresses):
    """"""
    max_cos_idx = []
    zip_code_list = []
    
    for crime in crime_coords:
        for idx, address in enumerate(address_coords):
            h_dist = haversine(crime[0], crime[1], address[0], address[1])
            if h_dist < 0.5:
                max_cos_idx.append(idx)
                zip_code_list.append(int(ny_addresses.loc[idx, 'POSTCODE']))
                break
        # this will be triggered if all ny addresses were checked and nothing within
        # the min diff was found
        if h_dist > 0.5:
            max_cos_idx.append(0)
            zip_code_list.append(0)
                
    return max_cos_idx, zip_code_list

### 2018

In [21]:
crime_coords = np.array(filtered_2018_df[['Latitude', 'Longitude']])
address_coords = np.array(ny_addresses[['LAT', 'LON']])

_, zips = get_crime_zips(crime_coords, address_coords, ny_addresses)

In [23]:
assert len(filtered_2018_df) == len(zips)

filtered_2018_df['zip_code'] = zips
filtered_2018_df.to_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'), index=False)

### 2017

In [33]:
crime_coords = np.array(filtered_2017_df[['Latitude', 'Longitude']])
address_coords = np.array(ny_addresses[['LAT', 'LON']])

_, zips = get_crime_zips(crime_coords, address_coords, ny_addresses)

In [35]:
assert len(filtered_2017_df) == len(zips)

filtered_2017_df['zip_code'] = zips
filtered_2017_df.to_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2017.csv'), index=False)

### 2019

In [36]:
crime_coords = np.array(filtered_2019_df[['Latitude', 'Longitude']])
address_coords = np.array(ny_addresses[['LAT', 'LON']])

_, zips = get_crime_zips(crime_coords, address_coords, ny_addresses)

In [37]:
assert len(filtered_2019_df) == len(zips)

filtered_2019_df['zip_code'] = zips
filtered_2019_df.to_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2019.csv'), index=False)

Things to do:

1. Append zip code to every crime

2. EDA/Summary stats on crime at the zip code level
    * total counts
    * counts by type
    * crime by month/day of week

3. Correlation between counts of crime and housing price
    * year-to-year and lagged?
    
    